In [70]:
!pip install transformers
!pip install sentence_transformers
!pip install datasets
!pip install accelerate
!pip install sentencepiece

In [77]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM ,TrainingArguments, AutoTokenizer  , Trainer,pipeline ,AutoModelForSequenceClassification,EarlyStoppingCallback , BertTokenizer , BertModel
from datasets import Dataset , load_dataset
import tensorflow as tf
import json
import ast
import numpy as np
import shutil  # Add this import statement at the top of your script

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import re
import torch
import torch.nn as nn

In [72]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [82]:
device

device(type='cuda')

In [83]:
torch.cuda.is_available()

True

In [84]:
data = pd.read_csv(r'/content/drive/MyDrive/train.csv')
data

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [85]:
# data.to_csv(f'/content/drive/MyDrive/train.csv',index = False)

In [86]:
listed = ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']

In [87]:
target = data[listed].values

In [88]:
target

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 0]])

In [89]:
# x= []
# for i in range (len(data)):
#   value = 1
#   matches = data.loc[i,listed].isin([value])
#   column_names = matches[matches].index.tolist()
#   n = str(column_names)
#   new_string = n.replace('[', '').replace(']', '')
#   x.append(new_string)

In [90]:

data['content'] = data['TITLE']+data['ABSTRACT']
data.drop(['ID','TITLE','ABSTRACT'],axis = 1,inplace=True)


In [91]:
# # to replact the place of column from 2 to 1
columns = data['content']
data.drop('content',axis = 1,inplace=True)
data.insert(0,'content',columns)

In [92]:
data

,content,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,Reconstructing Subject-Specific Effect Maps P...,1,0,0,0,0,0
1,Rotation Invariance Neural Network Rotation i...,1,0,0,0,0,0
2,Spherical polyharmonics and Poisson kernels fo...,0,0,1,0,0,0
3,A finite element approximation for the stochas...,0,0,1,0,0,0
4,Comparative study of Discrete Wavelet Transfor...,1,0,0,1,0,0
...,...,...,...,...,...,...,...
20967,Contemporary machine learning: a guide for pra...,1,1,0,0,0,0
20968,Uniform diamond coatings on WC-Co hard alloy c...,0,1,0,0,0,0
20969,Analysing Soccer Games with Clustering and Con...,1,0,0,0,0,0
20970,On the Efficient Simulation of the Left-Tail o...,0,0,1,1,0,0


In [93]:
# data['listed'].values

In [94]:
# data['listed']

In [95]:
max_len = 128
batch_size = 32
epochs = 2
lr = 1e-05

In [96]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [97]:

import torch.utils


class CustomDataset (torch.utils.data.Dataset):
    def __init__ (self , tokenizer, df,max_len ):
        self.tokenizer = tokenizer
        self.df = df
        self.max_len = max_len
        self.title = self.df['content']
        self.targets = self.df[listed].values
    def __len__(self):
        return len(self.title)
    def __getitem__(self,index):
        title = str(self.title[index])
        title = ''.join(title.split())

        inputs = self.tokenizer.encode_plus(title,
                                            None , add_special_tokens = True,
                                            max_length = self.max_len,
                                            truncation = True,
                                            padding = 'max_length',
                                            return_tensors = 'pt',
                                            return_attention_mask = True)
        return {'input_ids':inputs['input_ids'].flatten(),
                'attention_mask':inputs['attention_mask'].flatten(),
                'token_type_ids':inputs['token_type_ids'].flatten(),
                'targets': torch.FloatTensor(self.targets[index])
}


In [98]:
train_size = .8
train_df = data.sample(frac=train_size,random_state=200).reset_index(drop=True)
val_df = data.drop(train_df.index).reset_index(drop=True)

In [99]:
val_df

,content,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,Fundamental limits of low-rank matrix estimati...,0,0,1,0,0,0
1,Scalable Gaussian Process Inference with Finit...,0,0,0,1,0,0
2,Integral representations and asymptotic behavi...,0,0,1,0,0,0
3,Efficient Mendler-Style Lambda-Encodings in Ce...,1,0,0,0,0,0
4,Super Jack-Laurent Polynomials Let $\mathcal{...,0,0,1,0,0,0
...,...,...,...,...,...,...,...
4189,Contemporary machine learning: a guide for pra...,1,1,0,0,0,0
4190,Uniform diamond coatings on WC-Co hard alloy c...,0,1,0,0,0,0
4191,Analysing Soccer Games with Clustering and Con...,1,0,0,0,0,0
4192,On the Efficient Simulation of the Left-Tail o...,0,0,1,1,0,0


In [100]:
train_data= CustomDataset(tokenizer,train_df,max_len)
val_data = CustomDataset(tokenizer,val_df,max_len)

In [101]:
import torch.utils.data


train_data = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True,num_workers=0)
val_data = torch.utils.data.DataLoader(val_data,batch_size=batch_size,shuffle=False,num_workers=0)

In [102]:
def load_checkpoint (load_cp , model , optimizer):
    checkpoint = torch.load(load_cp)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min = checkpoint['valid_loss_min']
    return model,optimizer,checkpoint['epoch'],valid_loss_min.item()
def save_path(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)


In [103]:
class BertClass(nn.Module):
    def __init__(self):
        super (BertClass,self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased',return_dict =True)
        self.dropout = nn.Dropout(.3)
        self.linear = nn.Linear(768,6)
    def forward (self,input_ids,attention_mask  ,token_type_ids ):
        output = self.bert_model(input_ids , attention_mask , token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output  =self.linear(output_dropout)
        return output
model = BertClass()
model.to(device)

BertClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [104]:
def loss_fn(output ,target):
    return torch.nn.BCEWithLogitsLoss()(output, target)
optimizer = torch.optim.Adam(params = model.parameters(),lr = lr)

In [ ]:
val_targets=[]
val_outputs=[]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_path(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_path(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model


In [17]:
check_point = f'/content/drive/MyDrive/curr_ckpt'
best_model_bath = f'/content/drive/MyDrive/best_model.pt'


In [ ]:
trained_model = train_model(epochs, train_data, val_data, model, optimizer, check_point, best_model_bath)


############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000655 	Average Validation Loss: 0.002062
Validation loss decreased (inf --> 0.002062).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000490 	Average Validation Loss: 0.001681
Validation loss decreased (0.002062 --> 0.001681).  Saving model ...
############# Epoch 2  Done   #############



In [56]:
example = val_df['content'][4]

In [ ]:
example

'Super Jack-Laurent Polynomials  Let $\\mathcal{D}_{n,m}$ be the algebra of the quantum integrals of the\ndeformed Calogero-Moser-Sutherland problem corresponding to the root system of\nthe Lie superalgebra $\\frak{gl}(n,m)$. The algebra $\\mathcal{D}_{n,m}$ acts\nnaturally on the quasi-invariant Laurent polynomials and we investigate the\ncorresponding spectral decomposition. Even for general value of the parameter\n$k$ the spectral decomposition is not simple and we prove that the image of the\nalgebra $\\mathcal{D}_{n,m}$ in the algebra of endomorphisms of the generalised\neigen-space is $k[\\varepsilon]^{\\otimes r}$ where $k[\\varepsilon]$ is the\nalgebra of the dual numbers the corresponding representation is the regular\nrepresentation of the algebra $k[\\varepsilon]^{\\otimes r}$.\n'

In [135]:
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=max_len,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [ ]:
model.eval()


BertClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
with torch.no_grad():
  input_ids =encodings['input_ids'].to(device ,dtype = torch.long)
  attention_mask = encodings['attention_mask'].to(device,dtype = torch.long)
  token_type_ids = encodings['token_type_ids'].to(device,dtype=torch.long)
  output = model(input_ids,attention_mask,token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  print(final_output)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.06330715864896774, 0.09704843908548355, 0.92328280210495, 0.031970106065273285, 0.020401915535330772, 0.028606818988919258]]
Mathematics


<ipython-input-130-f1a92b9ea384>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])


In [ ]:
(np.argmax(final_output))

2

In [ ]:
train_df.columns[1:][np.argmax(final_output)]

'Mathematics'

In [105]:
# to load model from pytorch
model = BertClass().to(device)

checkpoint = torch.load(f'/content/drive/MyDrive/best_model.pt')

# Load the state dictionaries
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [106]:
model

BertClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [110]:
with torch.no_grad():
  input_ids =encodings['input_ids'].to(device ,dtype = torch.long)
  attention_mask = encodings['attention_mask'].to(device,dtype = torch.long)
  token_type_ids = encodings['token_type_ids'].to(device,dtype=torch.long)
  output = model(input_ids,attention_mask,token_type_ids)
  final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
  print(final_output)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

[[0.05918653681874275, 0.10291198641061783, 0.9218056797981262, 0.0272146575152874, 0.02077033929526806, 0.016155175864696503]]
Mathematics


<ipython-input-110-f1a92b9ea384>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])


In [133]:
def evaluate_model( model,test_loader):
  model.eval()
  all_target =  []
  all_prediction = []
  with torch.no_grad():
    for batch in test_loader:
      input_ids = batch['input_ids'].to(device,dtype=torch.long)
      attention_mask = batch['attention_mask'].to(device,dtype = torch.long)
      token_type_ids= batch['token_type_ids'].to(device,dtype=torch.long)
      targets = batch['targets'].to(device, dtype=torch.float)
      output =model(input_ids,attention_mask,token_type_ids)
      predictions = torch.sigmoid(output).cpu().numpy()
      all_target.extend(targets.cpu().numpy())
      all_prediction.extend(predictions)
  all_target =np.array(all_target)
  all_prediction=np.round(np.array(all_prediction))
  accuracy = accuracy_score(all_target, all_prediction)
  precision = precision_score(all_target, all_prediction, average='weighted')
  recall = recall_score(all_target, all_prediction, average='weighted')
  f1 = f1_score(all_target, all_prediction, average='weighted')
  roc_auc = roc_auc_score(all_target, all_prediction, average='weighted')

  print(f'Accuracy: {accuracy:.4f}')
  print(f'Precision: {precision:.4f}')
  print(f'Recall: {recall:.4f}')
  print(f'F1 Score: {f1:.4f}')
  print(f'ROC AUC: {roc_auc:.4f}')


  return accuracy, precision, recall, f1, roc_auc

In [134]:
accuracy, precision, recall, f1, roc_auc= evaluate_model( model,val_data)


Accuracy: 0.6307
Precision: 0.8046
Recall: 0.7463
F1 Score: 0.7740
ROC AUC: 0.8363


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
